Alignment -> mencari susunan terbaik antara Target Sequence dan Query Sequence 

1. Global Alignment
Mencari alignment terbaik dari kedua sequence secara keseluruhan, biasanya untuk 2 sequence yg panjangnya sama kalau tidak sm harus diisi dengan gap

2. Local Alignment 
Mencari alignment terbaik dari sequence dari subsequence yg ada, bisa digunakana untuk 2 sequence yg panjangnya berbeda

Similiarity analysis -> Mencari skor kesamaan dari kedua sequence.

Match : nucleotida yg sama (|)
Mismatch : nucleotida yg beda (.)
Gap : nucleotida yg diberi jarak sehingga jadi gak ada pansangan (-)

In [1]:
%pip install BioPython

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from Bio.Seq import Seq

dnaA = Seq("CGATGTCA")
dnaB = Seq("ACGTGTAC")

In [7]:
#Global Alignment
from Bio import pairwise2
alignments = pairwise2.align.globalxx(dnaA, dnaB)
#berisi semua alignment terbaik dari kedua sequence

#View possible alignment
for alignment in alignments:
    print(alignment)
#untuk setiap alignment yang ada di alignments, kita akan print

#Format aligment -> method yg ada di lib pairwise
from Bio.pairwise2 import format_alignment
for alignment in alignments:
    print(format_alignment(*alignment))

#Score -> nilai kecocokan untuk alignment

Alignment(seqA='-CGATGTCA-', seqB='ACG-TGT-AC', score=6.0, start=0, end=10)
Alignment(seqA='-CGATGT-CA', seqB='ACG-TGTAC-', score=6.0, start=0, end=10)
-CGATGTCA-
 || ||| | 
ACG-TGT-AC
  Score=6

-CGATGT-CA
 || ||| | 
ACG-TGTAC-
  Score=6



In [9]:
#XX -> default score
#mx -> custom match & mismatch score
#ms -> custom match, mismatch, gap start, and gap extend score

#Custom Aligmnet
#Match +1
#Mismatch -1
#Gap start -0.5
#Gap extend -0.1 -> kalau (-)ny 2x

alignments = pairwise2.align.globalms(dnaA, dnaB, 1 , -1, -0.5, -0.1)
for alignment in alignments:
    print(format_alignment(*alignment))

-CGATGTCA-
 || ||| | 
ACG-TGT-AC
  Score=4

-CGATGT-CA
 || ||| | 
ACG-TGTAC-
  Score=4



Hamming Distance, Levenshtein Distance, dan Dot Plot
-> Cari skor perbedaan nukleotida dari kedua sequence

Hamming Distance (tiap mismatch akan tambah 1 point)
-> Mencari perbedaan dua sequence dengan panjang yg sama

Levenshtein Distance
-> Pengertian nukleotida paling sedikit untuk menyamakan kedua sequence

Dot Plot
-> membuat matrix nukleotida dan memunculkan grafik dan plot dari nukleotida yg sama dari dua sequence

In [10]:
#Hamming Distance 
def hamming(Seq1, Seq2):
    h = [] #list kosong

    #iterasi
    for x, y in zip(Seq1, Seq2):
        if(x != y):
            h.append((x,y))
    return len(h)

In [12]:
from Bio.Seq import Seq

dnaA = Seq("CGATGTCA")
dnaB = Seq("ACGTGTAC")

In [11]:
print(hamming(dnaA, dnaB))

5


In [13]:
%pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [14]:
from Levenshtein import distance
print(distance(dnaA, dnaB))

4


In [34]:
#Dot Plot

#Delta -> apakah necleotida itu match
def delta(x, y):
    if(x != y):
        return 1
    else:
        return 0
    
# M -> mencari total delta sesuai panjang k
# i = starting point 1, j = 2, k = panjang subsequence yg kita mau
def M(Seq1, Seq2, i, j, k):
    m = [] #list kosong

    for x, y in zip (Seq1[i:i+k], Seq2[j:j+k]):
        m.append(delta(x, y))
    return sum(m)

#makeMatrix -> untuk buat list 2 dimensi sbg matrix
def makeMatrix(seq1, seq2, k):
    m1 = []
    for i in range(len(seq1) - k + 1):
        m2 = []
        for j in range (len(seq2) - k + 1):
            m2.append(M(seq1, seq2, i, j, k))
        m1.append(m2)
    return m1

#plotMatrix
#nonblank karakter yg di print kalau terjadi persamaan
#blank karakter yg diprint kalo nukleotida berbeda
def plotMatrix(M, tolerance, seq1, seq2,
               nonblank=chr(0x25A0), blank = ' '):
    print('|' + seq2) # | -> 
    print('-' * (4 + len(seq2)))

    #label = tiap nekleotida dari seq1
    #row = 1 baris matrix td
    for label, row in zip(seq1, M):
        line = ''.join(nonblank if score < tolerance else blank 
                       for score in row)
        print(label + '|' + line)

#dotplot -> untuk nyatuin function kita
def dotplot(Seq1, Seq2, k=1, t=1):
    Matrix = makeMatrix(Seq1, Seq2, k)
    plotMatrix(Matrix, t, Seq1, Seq2)

In [35]:
dotplot(dnaA, dnaB)
#kalo A sm A akan print chr
#kalo beda akan print blank

|ACGTGTAC
------------
C| ■     ■
G|  ■ ■   
A|■     ■ 
T|   ■ ■  
G|  ■ ■   
T|   ■ ■  
C| ■     ■
A|■     ■ 
